In [505]:
import json
import time
import datetime
import shutil

try:
    from mal import *
    config.TIMEOUT = 20  # Import level config
except:
    print("MAL-Api nicht gefunden. Installieren mit 'pip install -U mal-api'")

In [506]:
def getAnime(malid):
    maldata = None
    timeout = 1
    while maldata is None:
        try:
            maldata = Anime(malid, timeout=10)
        except Exception as e:
            print(f"Fehler: {e}  Versuche nochmal in {int(timeout)} sec..")
            time.sleep(timeout)
            timeout=timeout*1.3
    return maldata

In [524]:
def list_load():
    try:
        with open('archive.json', encoding="utf-8") as f:
            data = json.loads(f.read())
        return data
    except:
        with open('archive.json', encoding="utf-8") as f:
            data = json.loads(f.read()[f.read().index("["):])
        return data

In [516]:
def list_cleanup(data, mal=False):
    clean = []
    clean_w = [] #watchlist at the end or beginning
    for i in range(len(data)):
        old = data[i]
        try:
            if mal: 
                a=1/0 #go to except
            if min([len(s) for s in list(old.values())])!=0:
                a=1/0
            tmp = {"index":old["index"], 
                   "malid":old["malid"],
                   "title":old["title"],
                   "titen":old["titen"],
                   "titjp":old["titjp"],
                   "imurl":old["imurl"],
                   "semes":old["semes"],
                   "progr":old["progr"],
                   "vdate":old["vdate"],
                   "statu":old["statu"]
                   }
        except Exception as es:
            if not mal: 
                print(f"Ungültiger Eintrag (id:{old['malid']}) entdeckt, lade neu von MAL..")
            else:
                print("Lade von MAL..")
            
            maldata = getAnime(old["malid"])
                    
            tmp = {"index":old["index"], 
                   "malid":old["malid"],
                   "title":maldata.title,
                   "titen":maldata.title_english,
                   "titjp":maldata.title_japanese,
                   "imurl":maldata.image_url.replace(".jpg","l.jpg").replace("ll.jpg","l.jpg"), #get large picture
                   "semes":old["semes"],
                   "progr":("00"+old["progr"][:2].replace("/",""))[-2:]+"/"+("00"+str(maldata.episodes).replace("None","??"))[-2:],
                   "vdate":old["vdate"],
                   "statu":old["statu"]
                   }
        print("")
        print(f"Eintrag {tmp['index']}:")
        print(tmp['title'])
        print(tmp["titen"])
        print(tmp["titjp"])
        print(tmp["statu"], tmp["progr"])
        print("")
        print("")
        if data[i]["statu"]!="W": 
            clean.append(tmp)
        else:
            clean_w.append(tmp)
    data = clean_w+clean
    return data

In [517]:
def list_statusupdate(data, flag_needsort=False):

    print("Update-Datum eingeben mit Format: 31.01.2022 >> '3101[2022]' (Punkte und Jahr optional)")
    date = input()

    if len(date)>=4 and len(date)<=6:
        date += str(datetime.datetime.now().year)
    if len(date)==8:
        date = date[:2]+"."+date[2:4]+"."+date[4:]

    print("Datum: "+date)
    print("")
    print("STATUS UPDATEN: (Eingabe optional)")
    print("")

    for i in range(len(data)):
        if data[i]["statu"]=="W":
            print("Titel:", data[i]["title"])
            print("Folge:", data[i]["progr"])
            inc = input("increment:  +")
            sta = input("status change (W/F/D):")
            print("")
            if inc!="":
                try:
                    data[i]["progr"]=("00"+str(int(data[i]["progr"][:2])+int(inc)))[-2:]+data[i]["progr"][2:]
                except:
                    print(f"ERROR: '{data[i]['progr']}' + '{inc}' kein gültiges increment")
                    print("")
            if sta!="":
                if sta in ["W","F","D"]:
                    data[i]["statu"] = sta
                    flag_needsort = True
                else:
                    print(f"ERROR: '{sta}' kein gültiger Status")
                    print("")
            print("")

    print("Programm:") 

    ids = []
    for i in range(len(data)):
        if data[i]["statu"]=="W":
            ids.append(i)

    for i,j in zip(ids,range(len(ids))):
         print(j, "--", data[i]["title"])
    print("Neue Reihenfolge eingeben mit Format '0123': ")
    rf = input()
    if rf != "": #falls neue reihenfolge
        tmp = []
        if sorted(rf) == [str(k) for k in range(len(ids))]:
            for i in rf:
                tmp.append(data[ids[int(i)]])
            for i in range(len(ids)):
                data[sorted(ids)[i]] = tmp[i]
        else:
            print("")
            print("ERROR: Reihenfolge ist ungültig!")

    print("")
    print("FERTIG")
    if flag_needsort:
        print("Sortieren notwendig...")
        time.sleep(3)
        list_cleanup(mal = False)
    return data

In [518]:
def list_hinzufuegen(data):
    print("NEUEN ANIME HINZUFÜGEN/ALTEN WIEDERBELEBEN:")
    malid = input("MAL-ID (oder Titel für Suche):")
    maldata = None
    try:
        maldata = Anime(malid)
    except:
        print("")
        print("ID ungültig. Suchergebnisse:")
        try:
            search = AnimeSearch(malid)
            for i in range(min(15,len(search.results))):
                print(f'{(" "+str(i+1))[-2:]} -- {search.results[i].title} ({search.results[i].type})')
            i = input("Position: ")
            malid = search.results[int(i)-1].mal_id
            maldata = Anime(malid, timeout=10)
        except:
            print("")
            print("Suche nicht erfolgreich.")

    if maldata != None:
        print("")
        flag = False
        for i in range(len(data)):
            if data[i]["malid"] == malid:
                flag = True
                index = i
        if flag:
            print(f"Anime schon vorhanden. '{maldata.title}' wird wieder aktiviert.")
            data[i]["statu"] = "W"
        else:
            print(f"ID okay. '{maldata.title}' wird hinzugefügt")
            #semester
            if datetime.datetime.now().month in [4,5,6,7,8,9]:
                ts = str(datetime.datetime.now().year)[-2:]+"S"
            else:
                if datetime.datetime.now().month in [10,11,12]:
                    ts = str(datetime.datetime.now().year)[-2:]+"W"
                else:
                    ts = str(datetime.datetime.now().year-1)[-2:]+"W"
            #neuer eintrag
            tmp = {"index":len(data)+1, 
                   "malid":malid,
                   "title":maldata.title,
                   "titen":maldata.title_english,
                   "titjp":maldata.title_japanese,
                   "imurl":maldata.image_url,
                   "semes":ts,
                   "progr":"00/"+str(maldata.episodes).replace("None","??"),
                   "vdate":"--",
                   "statu":"W"
                   }
            data.append(tmp)

        print("")
        print("Statusupdate notwendig...")
        time.sleep(3)
        list_statusupdate(data, flag_needsort=True)
    return data

In [528]:
def list_savejson(data):
    #backup
    shutil.copy('archive.json', 'backup/archive'+datetime.datetime.now().isoformat()[:19].replace(":","-")+'.json')
    #save
    with open("archive.json", "w", encoding="utf-8") as outfile:
        json.dump(data, outfile, indent = 4)

In [531]:
data = list_load()

In [532]:
data = list_cleanup(data, mal=True)

Lade von MAL..

Eintrag 1:
Chuunibyou demo Koi ga Shitai! Ren
Love, Chunibyo & Other Delusions!: Heart Throb
中二病でも恋がしたい！戀
F 12/12


Lade von MAL..

Eintrag 2:
Kokoro Connect
Kokoro Connect
ココロコネクト
F 17/13


Lade von MAL..

Eintrag 3:
Fate/Zero
Fate/Zero
フェイト/ゼロ
F 13/13


Lade von MAL..

Eintrag 4:
Fate/Zero 2nd Season
Fate/Zero Season 2
フェイト/ゼロ 2ndシーズン
F 12/12


Lade von MAL..

Eintrag 5:
Danshi Koukousei no Nichijou
Daily Lives of High School Boys
男子高校生の日常
F 12/12


Lade von MAL..

Eintrag 6:
Zankyou no Terror
Terror in Resonance
残響のテロル
F 11/11


Lade von MAL..

Eintrag 7:
Tokyo Ghoul
Tokyo Ghoul
東京喰種-トーキョーグール-
F 12/12


Lade von MAL..

Eintrag 8:
Kaichou wa Maid-sama!
Maid Sama!
会長はメイド様!
F 26/26


Lade von MAL..

Eintrag 9:
Sidonia no Kishi
Knights of Sidonia
シドニアの騎士
F 12/12


Lade von MAL..

Eintrag 10:
Shinsekai yori
From the New World
新世界より
F 25/25


Lade von MAL..

Eintrag 11:
Girls & Panzer
Girls & Panzer
ガールズ&パンツァー
F 12/12


Lade von MAL..

Eintrag 12:
Gekkan Shoujo Nozaki-kun


Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 8 sec..

Eintrag 83:
Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen
Kaguya-sama: Love is War
かぐや様は告らせたい～天才たちの恋愛頭脳戦～
F 12/12


Lade von MAL..

Eintrag 84:
Kaguya-sama wa Kokurasetai?: Tensai-tachi no Renai Zunousen
Kaguya-sama: Love is War Season 2
かぐや様は告らせたい？～天才たちの恋愛頭脳戦～
F 12/12


Lade von MAL..

Eintrag 85:
Black Lagoon: The Second Barrage
Black Lagoon: The Second Barrage
BLACK LAGOON The Second Barrage
F 12/12


Lade von MAL..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 1 sec..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 1 sec..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 1 sec..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 2 sec..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 2 sec..
Fehler: 'NoneType' object is not subscriptable  Versuche nochmal in 3 sec..
Fehler: 'NoneType' object is no

In [533]:
list_savejson(data)